In [2]:
# !pip install virtualenv
# !virtualenv /content/notebooks/cs5228
# ! source /content/notebooks/cs5228/bin/activate && pip install pandas plotly scikit-learn
%load_ext autoreload
%autoreload 2
import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# sys.path.append('/content/notebooks/cs5228/lib/python3.10/site-packages')

In [3]:
import pandas as pd
import plotly
from data_processing import process_data, clean_data, cluster_rental_location

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
pd.options.plotting.backend = "plotly"

In [5]:
DATA_DIR = './data'
train = os.path.join(DATA_DIR, 'train.csv')
test = os.path.join(DATA_DIR, 'test.csv')

train_df, test_df = pd.read_csv(train), pd.read_csv(test)
test_df['flat_model'].unique(), train_df['flat_model'].unique()

(array(['improved', 'model a', 'new generation', 'premium apartment',
        'dbss', 'standard', 'model a2', 'simplified', '2-room',
        'maisonette', 'apartment', 'model a-maisonette', 'type s1',
        'type s2', 'adjoined flat', 'premium maisonette',
        'premium apartment loft', 'terrace'], dtype=object),
 array(['new generation', 'improved', 'apartment', 'premium apartment',
        'simplified', 'model a', 'standard', 'dbss', 'model a2',
        'maisonette', 'model a-maisonette', 'type s1', 'adjoined flat',
        'type s2', 'premium apartment loft', '2-room',
        'premium maisonette', 'terrace', '3gen'], dtype=object))

## Check null data

In [8]:
train_df.isnull().any()

rent_approval_date     False
town                   False
block                  False
street_name            False
flat_type              False
flat_model             False
floor_area_sqm         False
furnished              False
lease_commence_date    False
latitude               False
longitude              False
elevation              False
subzone                False
planning_area          False
region                 False
monthly_rent           False
dtype: bool

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rent_approval_date   60000 non-null  object 
 1   town                 60000 non-null  object 
 2   block                60000 non-null  object 
 3   street_name          60000 non-null  object 
 4   flat_type            60000 non-null  object 
 5   flat_model           60000 non-null  object 
 6   floor_area_sqm       60000 non-null  float64
 7   furnished            60000 non-null  object 
 8   lease_commence_date  60000 non-null  int64  
 9   latitude             60000 non-null  float64
 10  longitude            60000 non-null  float64
 11  elevation            60000 non-null  float64
 12  subzone              60000 non-null  object 
 13  planning_area        60000 non-null  object 
 14  region               60000 non-null  object 
 15  monthly_rent         60000 non-null 

In [10]:
train_df.furnished.unique()  # removed ?

array(['yes'], dtype=object)

## Clean the data

In [11]:
train_df_cleaned = clean_data(train_df)
train_df_cleaned = process_data(train_df_cleaned)
train_df_cleaned

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent,type_model,model_digit,type_digit,psqm,date,age
0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600,new generation 3-room,10,1,23.880597,1.630426e+09,14123
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250,new generation 4-room,10,2,24.456522,1.651334e+09,16191
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900,improved 3-room,5,1,28.358209,1.664554e+09,18901
3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850,apartment executive,3,4,19.127517,1.627747e+09,10439
4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100,improved 3-room,5,1,30.882353,1.667232e+09,18567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2021-09,ang mo kio,441,ang mo kio avenue 10,3-room,new generation,67.0,1979,1.366050,103.854168,chong boon,ang mo kio,north-east region,2200,new generation 3-room,10,1,32.835821,1.630426e+09,15584
59996,2023-04,bukit merah,95a,henderson road,4-room,model a,83.0,2019,1.286493,103.821434,henderson hill,bukit merah,central region,4100,model a 4-room,7,2,49.397590,1.680278e+09,1551
59997,2022-06,tampines,862a,tampines street 83,5-room,improved,122.0,1988,1.355064,103.936507,tampines west,tampines,east region,2250,improved 5-room,5,3,18.442623,1.654013e+09,12570
59998,2023-01,bedok,67,bedok sth avenue 3,5-room,standard,123.0,1977,1.318974,103.944076,bedok south,bedok,east region,4700,standard 5-room,15,3,38.211382,1.672502e+09,16801


## Attrbute Distribution

In [12]:
train_df_cleaned.columns

Index(['rent_approval_date', 'town', 'block', 'street_name', 'flat_type',
       'flat_model', 'floor_area_sqm', 'lease_commence_date', 'latitude',
       'longitude', 'subzone', 'planning_area', 'region', 'monthly_rent',
       'type_model', 'model_digit', 'type_digit', 'psqm', 'date', 'age'],
      dtype='object')

In [14]:
train_df_cleaned.plot.hist(x='psqm')

In [16]:
train_df.hist(x='monthly_rent')

### Check column values

In [60]:
train_df_cleaned['flat_model'].unique(), len(train_df_cleaned['flat_model'].unique())
# train_df_cleaned[train_df_cleaned['flat_model'] == '2-room']

(array(['new generation', 'improved', 'apartment', 'premium apartment',
        'simplified', 'model a', 'standard', 'dbss', 'model a2',
        'maisonette', 'model a-maisonette', 'type s1', 'adjoined flat',
        'type s2', 'premium apartment loft', '2-room',
        'premium maisonette', 'terrace', '3gen'], dtype=object),
 19)

In [10]:
train_df_cleaned['flat_type'].unique(), len(train_df_cleaned['flat_type'].unique())

(array(['3-room', '4-room', 'executive', '5-room', '2-room'], dtype=object), 5)

In [11]:
train_df_cleaned['type_model'].unique(), len(train_df_cleaned['type_model'].unique())

(array(['new generation 3-room', 'new generation 4-room',
        'improved 3-room', 'apartment executive',
        'premium apartment executive', 'premium apartment 5-room',
        'simplified 4-room', 'model a 4-room', 'improved 5-room',
        'model a 3-room', 'standard 3-room', 'simplified 3-room',
        'dbss 4-room', 'model a2 4-room', 'improved 4-room',
        'premium apartment 4-room', 'model a 2-room',
        'maisonette executive', 'improved 2-room', 'standard 5-room',
        'model a 5-room', 'premium apartment 3-room', 'dbss 3-room',
        'dbss 5-room', 'model a-maisonette 5-room', 'type s1 4-room',
        'adjoined flat executive', 'type s2 5-room', 'standard 4-room',
        'adjoined flat 5-room', 'premium apartment loft 4-room',
        'standard 2-room', 'adjoined flat 4-room', '2-room 2-room',
        'premium maisonette executive', 'premium apartment 2-room',
        'terrace 3-room', 'terrace 4-room', '3gen 5-room'], dtype=object),
 39)

## Average Price across time (overall)

In [13]:
trained_df_group_avg = train_df_cleaned.groupby('rent_approval_date').mean(numeric_only=True).reset_index()

In [32]:
trained_df_group_avg.plot.line(x='rent_approval_date', y='psqm', title='time vs. per-square-meter price', width=800, height=500)


## Visualize the location cluster (First Glance)

In [15]:
data = train_df[['latitude', 'longitude']].to_dict(orient='list')
data.keys()

dict_keys(['latitude', 'longitude'])

In [16]:
def show_cluster_location(data, n_clusters):

    # df = pd.DataFrame(data)

    # # Number of clusters (change this based on your requirement)

    # # KMeans clustering
    # kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df[['latitude', 'longitude']])
    # df['cluster'] = kmeans.labels_
    df = cluster_rental_location(train_df, n_clusters)[['latitude', 'longitude', 'cluster']]
    # Plotting on map using Plotly with Mapbox
    fig = px.scatter_mapbox(df,
                            lat='latitude',
                            lon='longitude',
                            color='cluster',
                            zoom=10,  # Adjust zoom level as needed
                            mapbox_style="open-street-map",  # Use open-street-map style
                            color_continuous_scale=px.colors.sequential.Plasma,
                            title="KMeans Clustering of rented house")

    fig.show()

In [17]:
show_cluster_location(data=data, n_clusters=6)

/Users/lgy/opt/anaconda3/envs/analysis/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/lgy/opt/anaconda3/envs/analysis/lib/python3.8/site-packages/threadpoolctl.py:1019: RuntimeWarning:

libc not found. The ctypes module in Python 3.8 is maybe too old for this OS.



variance by cluster

In [18]:
clustered_df = cluster_rental_location(train_df, 6)

/Users/lgy/opt/anaconda3/envs/analysis/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [19]:
clustered_df['monthly_rent'].std()

714.9104679238307

In [20]:
clustered_df.groupby('cluster')['monthly_rent'].std()

cluster
0    648.552476
1    695.870302
2    705.253152
3    682.160297
4    840.744522
5    678.242028
Name: monthly_rent, dtype: float64

In [21]:
clustered_df.groupby('town')['monthly_rent'].std()

town
ang mo kio         642.344210
bedok              641.510567
bishan             739.559310
bukit batok        655.718657
bukit merah        830.599794
bukit panjang      683.366077
bukit timah        866.126153
central            966.797551
choa chu kang      661.580163
clementi           727.075709
geylang            708.573146
hougang            663.601764
jurong east        728.884426
jurong west        707.557602
kallang/whampoa    714.371025
marine parade      694.459191
pasir ris          689.280575
punggol            695.336359
queenstown         861.933204
sembawang          653.665074
sengkang           674.009024
serangoon          672.455401
tampines           691.754444
toa payoh          702.513826
woodlands          647.419316
yishun             642.572235
Name: monthly_rent, dtype: float64

In [22]:
clustered_df.groupby('region')['monthly_rent'].std()

region
central region       799.174469
east region          680.710535
north region         648.552476
north-east region    675.303569
west region          700.541628
Name: monthly_rent, dtype: float64

## Analysis For different type

In [23]:
train_df_cleaned.columns

Index(['rent_approval_date', 'town', 'block', 'street_name', 'flat_type',
       'flat_model', 'floor_area_sqm', 'lease_commence_date', 'latitude',
       'longitude', 'subzone', 'planning_area', 'region', 'monthly_rent',
       'type_model', 'psqm', 'age', 'regional_psqm', 'town_psqm'],
      dtype='object')

In [35]:
def plot_psqm_trend(target_attr):
  trained_df_group_avg = train_df_cleaned.groupby(['rent_approval_date', target_attr]).mean(numeric_only=True).reset_index()
  return trained_df_group_avg.pivot(index='rent_approval_date', columns=target_attr, values='psqm').plot.line(title=f'{target_attr} per squre meter price', width=800, height=500)

In [36]:
plot_psqm_trend('region').show()

A useful feature could be the region psqm average, it capture the trend and region information

In [37]:
plot_psqm_trend('town').show()

Even though region price and overall price presents a strong raising singal, however, the price with town has more occilation. It may suggest that some other factor during the time period affecting the price as well.

per square meter of town could also be a useful feature 

## Price for type

In [39]:
train_df_cleaned.groupby('flat_model')['monthly_rent'].std()

flat_model
2-room                     696.905562
3gen                      1590.859307
adjoined flat             1138.741660
apartment                  721.599062
dbss                       712.651007
improved                   716.086106
maisonette                 765.423404
model a                    710.757682
model a-maisonette         828.086023
model a2                   615.333783
new generation             604.034005
premium apartment          704.108075
premium apartment loft     987.631848
premium maisonette        1747.212065
simplified                 646.724173
standard                   772.987472
terrace                   1441.353531
type s1                    971.717010
type s2                    912.045003
Name: monthly_rent, dtype: float64

In [40]:
train_df_cleaned.groupby('flat_type')['monthly_rent'].std()

flat_type
2-room       613.541815
3-room       560.914029
4-room       706.453729
5-room       729.186910
executive    739.069293
Name: monthly_rent, dtype: float64

In [45]:
train_df_cleaned.groupby('type_model')['monthly_rent'].std()

type_model
2-room 2-room                     696.905562
3gen 5-room                      1590.859307
adjoined flat 4-room              922.632104
adjoined flat 5-room             1202.942820
adjoined flat executive          1123.387734
apartment executive               721.599062
dbss 3-room                       544.954509
dbss 4-room                       638.692633
dbss 5-room                       738.781916
improved 2-room                   499.009063
improved 3-room                   542.091337
improved 4-room                   662.964485
improved 5-room                   721.775298
maisonette executive              765.423404
model a 2-room                    648.130293
model a 3-room                    596.332804
model a 4-room                    709.708935
model a 5-room                    668.207096
model a-maisonette 5-room         828.086023
model a2 4-room                   615.333783
new generation 3-room             532.448167
new generation 4-room             653.983416

In [50]:
train_df_cleaned.groupby(['rent_approval_date', 'type_model'])['monthly_rent'].std().mean()

490.39287494480385

In [49]:
train_df_cleaned.groupby(['rent_approval_date', 'flat_model'])['monthly_rent'].std().mean()

540.2943794631858

In [48]:
train_df_cleaned.groupby(['rent_approval_date', 'flat_type'])['monthly_rent'].std().mean()

521.1401596538021

In [58]:
train_df_cleaned.join(pd.get_dummies(train_df_cleaned['flat_type']).astype(int))

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,...,type_model,psqm,age,regional_psqm,town_psqm,2-room,3-room,4-room,5-room,executive
0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,103.738630,...,new generation 3-room,23.880597,14123,27.233545,27.915301,0,1,0,0,0
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,...,new generation 4-room,24.456522,16191,27.057755,29.456489,0,0,1,0,0
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,...,improved 3-room,28.358209,18901,33.891545,32.040421,0,1,0,0,0
3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,103.962894,...,apartment executive,19.127517,10439,27.057755,22.531163,0,0,0,0,1
4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,103.863341,...,improved 3-room,30.882353,18567,33.891545,33.333155,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2021-09,ang mo kio,441,ang mo kio avenue 10,3-room,new generation,67.0,1979,1.366050,103.854168,...,new generation 3-room,32.835821,15584,27.838756,30.492336,0,1,0,0,0
59996,2023-04,bukit merah,95a,henderson road,4-room,model a,83.0,2019,1.286493,103.821434,...,model a 4-room,49.397590,1551,33.891545,35.213157,0,0,1,0,0
59997,2022-06,tampines,862a,tampines street 83,5-room,improved,122.0,1988,1.355064,103.936507,...,improved 5-room,18.442623,12570,27.057755,26.820733,0,0,0,1,0
59998,2023-01,bedok,67,bedok sth avenue 3,5-room,standard,123.0,1977,1.318974,103.944076,...,standard 5-room,38.211382,16801,27.057755,29.456489,0,0,0,1,0
